## Database Engineer Capstone Week 3

***

### Import Libraries

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

import datetime
from datetime import datetime, timedelta
import scipy.stats


import mysql.connector as connector

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
# #sets the default autosave frequency in seconds
#pip install jupyter_contrib_nbextensions
# sns.set_style('dark')
# sns.set(font_scale=1.2)

# plt.rc('axes', titlesize=9)
# plt.rc('axes', labelsize=14)
# plt.rc('xtick', labelsize=12)
# plt.rc('ytick', labelsize=12)

# pd.set_option('display.max_columns',None)
# #pd.set_option('display.max_rows',None)
# pd.set_option('display.width', 1000)
# pd.option_context('float_format','{:.2f}'.format)

# random.seed(0)
# np.random.seed(0)
# np.set_printoptions(suppress=True)

## Load Data from SQL database

Your first step is to import the connector module, enter your user details and connect with the database (Hint: you can use an alias when importing the module).

### MySQL

In [42]:
#Create a connection

mydb = connector.connect(
  user="root",
  passwd="password",
  database="LittleLemonCoursera"
)

In [29]:
print(mydb)

In [30]:
mycursor = mydb.cursor()

In [31]:
mycursor

In [32]:
mydb.reconnect() #Reconnect cursor

In this second task, you now need to query the database to show all tables within the database. 

In [43]:
pd.read_sql_query('SHOW tables', mydb)

,Tables_in_littlelemoncoursera
0,booking
1,customer
2,delivery
3,menu
4,orders
5,staff


In [34]:
pd.read_sql_query("SHOW columns FROM booking", mydb)

,Field,Type,Null,Key,Default,Extra
0,BookingID,b'varchar(20)',NO,PRI,None,
1,Date,b'datetime',NO,,None,
2,TableNumber,b'int',NO,,None,


An alternate way to learn the same information would be to use the DESCRIBE function. The syntax is:

In [35]:
pd.read_sql_query("DESCRIBE booking", mydb)

,Field,Type,Null,Key,Default,Extra
0,BookingID,b'varchar(20)',NO,PRI,None,
1,Date,b'datetime',NO,,None,
2,TableNumber,b'int',NO,,None,


DELIMITER $$
DROP PROCEDURE IF EXISTS GetMaxQuantity;
CREATE PROCEDURE GetMaxQuantity()
BEGIN
	SELECT MAX(orders.Quantity)
	FROM orders;
END$$

DELIMITER $$
CREATE PROCEDURE AddBooking (IN BookingID INT, IN TableNumber INT, IN Date DATE)
BEGIN
	INSERT INTO booking (bookingid, tablenumber, date) VALUES (BookingID, TableNumber, Date); 
END$$

DELIMITER $$
CREATE PROCEDURE ManageBooking(Table_Number INT)
BEGIN
	SELECT *
	FROM booking
	WHERE TableNumber = Table_Number;
END$$

DELIMITER $$
DROP PROCEDURE IF EXISTS UpdateBooking;
CREATE PROCEDURE UpdateBooking (IN Booking_ID INT, IN Table_Number INT, IN Date DATE)
BEGIN
	SET SQL_SAFE_UPDATES = 0;
	UPDATE booking 
    SET Date = Date,  TableNumber = Table_Number
    WHERE BookingID = Booking_ID;
    SET SQL_SAFE_UPDATES = 1;
END$$

DELIMITER $$
DROP PROCEDURE IF EXISTS CancelBooking;
CREATE PROCEDURE CancelBooking(IN Booking_ID INT)
BEGIN
	SET SQL_SAFE_UPDATES = 0;
	DELETE FROM booking WHERE BookingID = Booking_ID;
    SET SQL_SAFE_UPDATES = 1;
END$$

-- Sample TEST
CALL CancelBooking('1');
CALL ManageBooking(5);
CALL UpdateBooking('1',5,'2023-07-11');
CALL AddBooking('1',5,'2023-07-10');


For the third and final task, Little Lemon need you to return specific details from your database. They require the full name and contact details for every customer that has placed an order greater than $60 for a promotional campaign. 

In [36]:
pd.read_sql_query("SELECT * FROM customer", mydb)

,CustomerID,CustomerName,ContactNumber,Email,City,Country,PostalCode,CountryCode,Customercol


In [37]:
pd.read_sql_query("SELECT * FROM orders", mydb)

,OrderID,BookingID,MenuID,CustomerID,DeliveryID,StaffID,OrderDate,Quantity,Cost,Sales,Discount


In [38]:
pd.read_sql_query("""SELECT customer.FullName, customer.ContactNumber FROM customer
                    INNER JOIN orders
                    ON customer.CustomerID = orders.Customer_CustomerID
                    WHERE orders.TotalCost > 60
                    """, mydb)

DatabaseError: Execution failed on sql 'SELECT customer.FullName, customer.ContactNumber FROM customer
                    INNER JOIN orders
                    ON customer.CustomerID = orders.Customer_CustomerID
                    WHERE orders.TotalCost > 60
                    ': 1054 (42S22): Unknown column 'customer.FullName' in 'field list'

The cloned repository contains a procedure called GetMaxQuantity(). Call this procedure and verify 

In [44]:
pd.read_sql_query("""CALL GetMaxQuantity();""", mydb)

,MAX(orders.Quantity)
0,None


Call the ManageBooking() procedure by passing the appropriate parameters. First with an available table number, then with one that has already been reserved.

In [40]:
pd.read_sql_query("""CALL ManageBooking(5);""", mydb)

OperationalError: MySQL Connection not available.

Create an SQL statement that calls the AddBooking() procedure

Call the cursor .execute() method using the above parameters. Print out the result using the cursor .fetchall() method.

In [16]:
pd.read_sql_query("""CALL AddBooking('1',5,'2023-07-10');""", mydb)

OperationalError: MySQL Connection not available.

In [16]:
mycursor.fetchall()

InterfaceError: No result set to fetch from

Create a SQL statement that calls the UpdateBooking() procedure

In [17]:
pd.read_sql_query("""CALL UpdateBooking('1',5,'2023-07-11');""", mydb)

OperationalError: MySQL Connection not available.

In [18]:
mycursor.fetchall()

InterfaceError: No result set to fetch from

Create a SQL statement that calls the CancelBooking() procedure

In [19]:
pd.read_sql_query("""CALL CancelBooking('1');""", mydb)

OperationalError: MySQL Connection not available.

In [20]:
mycursor.fetchall()

InterfaceError: No result set to fetch from